In [183]:
require 'cudnn'
require 'inn'
require 'image'
require 'dcnn'
Plot = require 'itorch.Plot'

-- Loads the mapping from net outputs to human readable labels
function load_synset()
  local file = io.open 'synset_words.txt'
  local list = {}
  while true do
    local line = file:read()
    if not line then break end
    table.insert(list, string.sub(line,11))
  end
  return list
end

-- Converts an image from RGB to BGR format and subtracts mean
function preprocess(im, img_mean)
  -- rescale the image
  local im3 = image.scale(im,224,224,'bilinear')*255
  -- RGB2BGR
  local im4 = im3:clone()
  im4[{1,{},{}}] = im3[{3,{},{}}]
  im4[{3,{},{}}] = im3[{1,{},{}}]

  -- subtract imagenet mean
  return im4 - image.scale(img_mean, 224, 224, 'bilinear')
end

-- Lua implementation of PHP scandir function
function scandir(directory)
    local i, t, popen = 0, {}, io.popen
    for filename in popen('ls -a "'..directory..'"'):lines() do
        if filename:match "%.JPEG$" then
            i = i + 1
            t[i] = filename

        end
    end
    return t, i
end


--fileDir = './test_n01443537'
fileDir = './n01443537'
fileList, total_files = scandir( fileDir )
true_label = 2



-- Setting up networks 
print '==> Loading network'

--net = torch.load('/home/yusheng/Workspace/DeepLearning/models/zeilerNet/zeilerNet.net')
net = torch.load('/usr/local/data/zeilerNet/zeilerNet.net')
--net:add(nn.Sigmoid())
net:cuda()
-- as we want to classify, let's disable dropouts by enabling evaluation mode
net:evaluate()


print '==> Loading synsets'
synset_words = load_synset()

print '==> Loading image and imagenet mean'
image_name = 'Goldfish3.jpg'
--image_name = 'lena.jpg'
--image_name='people2.jpg'
img_mean_name = 'ilsvrc_2012_mean.t7'

im = image.load(image_name)
img_mean = torch.load(img_mean_name).img_mean:transpose(3,1)

==> Loading network	


==> Loading synsets	


==> Loading image and imagenet mean	


In [184]:
function Find_diff_by_drop1( output_org )
    local topN_index = {}
    local n = 256
    for i=1, n do
        topN_index[i] = { idx=0, diff=nil }
    end

    for i=1, net:get(1):get(13).output:size(1) do
        local incomplete_fm = net:get(1):get(13).output:clone()
        incomplete_fm[i]:zero()
        local label_prob = {}
        label_prob = classifier:forward(incomplete_fm):view(-1):float()
        --local diff = output_org - NormalizedMax( sorted_prob )
        local diff = output_org - label_prob[true_label]
        --print( i.. '  '..diff )

        if topN_index[n].diff == nil then 
            topN_index[n].diff = diff
            topN_index[n].idx = i

            local idx = n-1
            while idx>=1 and topN_index[idx].diff == nil do
                topN_index[idx].diff = topN_index[idx+1].diff
                topN_index[idx].idx = topN_index[idx+1].idx
                topN_index[idx+1].diff = nil
                idx = idx - 1
            end
        
            if i==256 then
                for j=1,n-1 do 
                    for k=n,j+1,-1 do
                        -- print(k ..' '.. j )
                        if topN_index[k].diff > topN_index[k-1].diff then  
                            temp_diff = topN_index[k].diff
                            temp_idx = topN_index[k].idx
                            topN_index[k].diff = topN_index[k-1].diff
                            topN_index[k].idx = topN_index[k-1].idx
                            topN_index[k-1].diff = temp_diff
                            topN_index[k-1].idx = temp_idx
                        end
                    end
                end
            end
        else
            local temp_diff = nil
            local temp_idx = 0
            if n>1 then
                for j=1,n-1 do 
                    for k=n,j+1,-1 do
                        -- print(k ..' '.. j )
                        if topN_index[k].diff > topN_index[k-1].diff then  
                            temp_diff = topN_index[k].diff
                            temp_idx = topN_index[k].idx
                            topN_index[k].diff = topN_index[k-1].diff
                            topN_index[k].idx = topN_index[k-1].idx
                            topN_index[k-1].diff = temp_diff
                            topN_index[k-1].idx = temp_idx
                        end
                    end
                end
            end

            if topN_index[n].diff < diff then
                topN_index[n].diff = diff
                topN_index[n].idx = i
            end
        end
    end
--     for i=1,n do
--         print( 'Top'.. i ..' drop down : ' .. topN_index[i].idx .. ' Diff: ' .. topN_index[i].diff) 
--     end
    return topN_index
end

In [185]:
function Insertion( topN_index )
    local classification_curve = torch.Tensor(256)
    
    for top_n=1, 256 do
        local incomplete_fm = net:get(1):get(13).output:clone()
        for i=1,net:get(1):get(13).output:size(1) do
            local drop = true
            for j=1,top_n do
                if i == topN_index[j].idx then
                    drop = false
                end  
            end

            if drop == true then
                --print(i)
                incomplete_fm[i]:zero()
            end
        end

        local sorted_prob = {}
        local classess = {}
        sorted_prob, classes = classifier:forward(incomplete_fm):view(-1):float():sort(true)
        --output_org, average = NormalizedMax( sorted_prob )
        --local ratio_of_mean_error = (output_org - average)/average

--         for i=1,3 do
--             print('Top '..tostring(top_n)..' feature map classification: ', synset_words[classes[i]] .. ' ' .. sorted_prob[i] )
--         end
        
        --print(classes[1])
        if true_label == classes[1] then
            classification_curve[top_n]= sorted_prob[1] 
        else
            classification_curve[top_n] = 0
        end
    end
    
    return classification_curve
end

In [186]:
function Dropout( topN_index )
    local classification_curve = torch.Tensor(256)
    
    for top_n=1, 256 do
        local incomplete_fm = net:get(1):get(13).output:clone()
        for i=1,net:get(1):get(13).output:size(1) do
            local drop = false
            for j=1,top_n do
                if i == topN_index[j].idx then
                    drop = true
                end  
            end

            if drop == true then
                --print(i)
                --print('Drop neuron :'.. i)
                incomplete_fm[i]:zero()
            end
        end

        local sorted_prob = {}
        local classess = {}
        sorted_prob, classes = classifier:forward(incomplete_fm):view(-1):float():sort(true)
        --output_org, average = NormalizedMax( sorted_prob )
        --local ratio_of_mean_error = (output_org - average)/average

        --print('Top '..tostring(top_n)..' feature map classification: ', synset_words[classes[1]] .. ' ' .. sorted_prob[1] )

        --print(classes[1])
        if true_label == classes[1] then
            classification_curve[top_n]= sorted_prob[1] 
        else
            classification_curve[top_n] = 0
        end
    end
    
    return classification_curve
end

In [187]:
print '==> Preprocessing'
--print(fileList)
classifier = nn.Sequential()
classifier:add(net:get(2))
classifier:add(net:get(3))

point_index = torch.Tensor(256)
for i=1, 256 do
    point_index[i] = i
end

local specific_object_statistic = {}
for i=1, total_files do
    specific_object_statistic[i] = { file_name='nil', insertion_curve={}, dropout_curve={}, topN_index={}  }
end

timer = torch.Timer()
--for i=21, 21 do
for i=1, total_files do
    im = image.load( fileDir..'/'..fileList[i] )
    I = preprocess(im, img_mean)

    sorted_prob, classes = net:forward(I:cuda()):view(-1):float():sort(true)
    label_prob = net:forward(I:cuda()):view(-1):float()
    print('Top '..tostring(top_n)..' feature map classification: ', synset_words[classes[1]] .. ' ' .. sorted_prob[1] )
    print( label_prob[true_label] )
    
    print( i..' is finished')
    local topN_index = Find_diff_by_drop1( label_prob[true_label] )
    --print(topN_index )
    local insertion_curve = Insertion( topN_index )
    local dropout_curve = Dropout( topN_index )
    
    specific_object_statistic[i].file_name = fileList[i]
    specific_object_statistic[i].insertion_curve = insertion_curve
    specific_object_statistic[i].dropout_curve = dropout_curve
    specific_object_statistic[i].topN_index = topN_index
    --plot = Plot():line( point_index, insertion_curve,'red','Classification Curve'):legend(true):title('Insertion'):draw()
    --plot = Plot():line( point_index, dropout_curve,'red','Classification Curve'):legend(true):title('Dropout'):draw() 
end

torch.save('n01443537.t7', specific_object_statistic )
print('==> Time elapsed: ' .. timer:time().real .. ' seconds')

==> Preprocessing	


Top nil feature map classification: 	goldfish, Carassius auratus 14.475586891174	
14.475586891174	
1 is finished	


Top nil feature map classification: 	goldfish, Carassius auratus 25.032028198242	


25.032028198242	
2 is finished	


Top nil feature map classification: 	goldfish, Carassius auratus 12.869455337524	
12.869455337524	
3 is finished	


...

Top nil feature map classification: 	goldfish, Carassius auratus 14.671661376953	
14.671661376953	
1297 is finished	


Top nil feature map classification: 	goldfish, Carassius auratus 16.168018341064	
16.168018341064	
1298 is finished	


Top nil feature map classification: 	coral reef 15.277828216553	
13.572606086731	
1299 is finished	


Top nil feature map classification: 	goldfish, Carassius auratus 7.9457354545593	
7.9457354545593	
1300 is finished	


==> Time elapsed: 2190.9037089348 seconds	


In [142]:
img=image.load(fileDir..'/'..fileList[5])
itorch.image(img)

In [188]:
--test = torch.load('./specific_object_statistic.t7')
goldfish = torch.load('./n01443537.t7')
--print(test[1].topN_index)
-- for i=1, 50 do
--     plot = Plot():line( point_index, test[i].insertion_curve,'red','Classification Curve'):legend(true):title('Insertion '..i):draw()
--     --plot = Plot():line( point_index, test[i].dropout_curve,'red','Classification Curve'):legend(true):title('Dropout '..i):draw() 
-- end